# 보험 다모아 사이트 크롤링
- http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsIntro.knia
- 보장성 보험 데이터 크롤링
    - 질병 보험
    - 상해 보험
    - 운전자 보험
    - 어린이 보험
    - 재물 보험
    - 골프 보험
- requests 이용

## 1. 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 보장성 보험 데이터 가져오기 : http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsIntro.knia
url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C001" # 질병 보험
# url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C003" # 상해 보험
# url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C004" # 운전자 보험
# url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C005" # 어린이 보험
# url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C008" # 재물 보험
# url = "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=D001" # 골프 보험

response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")

In [3]:
# 데이터 선택
datas = dom.select(".div")
len(datas)

49

In [4]:
itmes = []    

for data in datas:
    item = {}
    item["company"] = data.select_one(".comp_logo").text.replace("\n", "")
    item["product"] = data.select_one("td:nth-child(3)").text
    item["total_cost"] = data.select_one(".total_cost").text
    item["age"] = data.select_one("td:nth-child(6)").text
    item["etc"] = data.select_one("td:nth-child(7)").text.replace("\n", "")
    
    # 보장명, 보장 금액
    guarantees = data.select(".item_list > li")
    for idx, guarantee in enumerate(guarantees):
        item["guarantee_" + str(idx)] = guarantee.select_one(".kind").text
        item["guarantee_" + str(idx) + "_cost"] = guarantee.select_one(".cost").text
    itmes.append(item)
        
itmes[0]

{'company': '교보라이프플래닛',
 'product': '(무)e입원비보험(순수보장형)',
 'total_cost': '1,440',
 'age': '19~60',
 'etc': '보장(담보) 총 2종  5년만기/전기납',
 'guarantee_0': '질병재해 입원보험금',
 'guarantee_0_cost': '1만원 (입원일수 1일당)',
 'guarantee_1': '특정 질병재해 입원보험금',
 'guarantee_1_cost': '2만원 (입원일수 1일당)'}

In [5]:
# 데이터 프레임으로 만들기
df = pd.DataFrame(itmes)
colums = ["company", "age", "product", "total_cost", "etc"]
df[colums].tail(2)

company    age                       product total_cost  \
47    동양생명  17~42    (무)수호천사온라인두근두근산모보험_2형(실속형)     48,300   
48    동양생명  17~42  (무)수호천사온라인두근두근산모보험_1형(프리미엄형)    168,000   

                                   etc  
47      보장담보 총3종, 1년만기, 일시납, 순수보장형, 여자  
48  보장담보 총6종, 1년만기, 일시납, 순수보장형, 여자....

In [7]:
urls = {"질병보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C001",
        "상해보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C003",
        "운전자보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C004",
        "어린이보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C005",
        "재물보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C008",
        "골프보험": "http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=D001",
        }

## 2. 크롤링 함수로 만들기

In [8]:
# 함수로 만들기
def guaranteed_insurance(urls):
    
    dfs = []
    
    def crawling(url):
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        datas = dom.select(".div")
        print(len(datas))
        return datas
        
    def select_datas(datas, name):
        itmes = []    

        for data in datas:
            item = {}
            item["name"] = name
            item["company"] = data.select_one(".comp_logo").text.replace("\n", "")
            item["product"] = data.select_one("td:nth-child(3)").text
            item["total_cost"] = data.select_one(".total_cost").text
            item["age"] = data.select_one("td:nth-child(6)").text
            item["etc"] = data.select_one("td:nth-child(7)").text.replace("\n", "")

            # 보장명, 보장 금액
            guarantees = data.select(".item_list > li")
            for idx, guarantee in enumerate(guarantees):
                item["guarantee_" + str(idx)] = guarantee.select_one(".kind").text
                item["guarantee_" + str(idx) + "_cost"] = guarantee.select_one(".cost").text
            itmes.append(item)
        return pd.DataFrame(itmes)
         
    for name, url in urls.items():
        print(name, url, end="\t")
        datas = crawling(url)
        df = select_datas(datas, name)
        dfs.append(df)
        
    return dfs

In [9]:
# 크롤링 함수 실행
dfs = guaranteed_insurance(urls)
len(dfs)

질병보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C001	49
상해보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C003	24
운전자보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C004	13
어린이보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C005	21
재물보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=C008	8
골프보험 http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsList.knia?menuId=D001	4


6

In [10]:
# 컬럼 선택하기
columns = ["name", "company", "product", "total_cost", "age", "etc"]

In [11]:
dfs[0][columns].tail(2)

name company                       product total_cost    age  \
47  질병보험    동양생명    (무)수호천사온라인두근두근산모보험_2형(실속형)     48,300  17~42   
48  질병보험    동양생명  (무)수호천사온라인두근두근산모보험_1형(프리미엄형)    168,000  17~42   

                                   etc  
47      보장담보 총3종, 1년만기, 일시납, 순수보장형, 여자  
48  보장담보 총6종, 1년만기, 일시납, 순수보장형, 여자....

In [12]:
result_df = pd.concat(dfs, axis=0, sort=True, ignore_index=True)[columns]
print(len(result_df))
result_df.tail(2)

119


name company         product total_cost    age  \
117  골프보험    KB손보  KB골프보험(다이렉트플랜)     29,260  20~69   
118  골프보험    롯데손보           홀인원보험     91,260  0~100   

                                  etc  
117  보장(담보) 총3종/1년 단기소멸성/ 기본형/표준형/고급형  
118          1년단기보험 (소멸성), 실속형/고급형 플랜

In [13]:
# csv 파일로 저장
result_df.to_csv("guaranteeIns.csv", index=False)

In [14]:
# csv 파일로 로드
result_df = pd.read_csv("guaranteeIns.csv")
result_df.tail(2)

name company         product total_cost    age  \
117  골프보험    KB손보  KB골프보험(다이렉트플랜)     29,260  20~69   
118  골프보험    롯데손보           홀인원보험     91,260  0~100   

                                  etc  
117  보장(담보) 총3종/1년 단기소멸성/ 기본형/표준형/고급형  
118          1년단기보험 (소멸성), 실속형/고급형 플랜